# Task T2.13: Voting Classifier (Ensemble)
In this task, we implemented an ensemble learning approach by combining the top three supervised learning models developed during the research phase. The goal is to leverage the strengths of multiple algorithms to improve classification robustness and accuracy.


# 1. Methodology

Preprocessing & Scaling: To ensure consistency across distance-based models like SVM and tree-based models like XGBoost, we applied StandardScaler to the processed dataset.


Model Selection: Based on the individual performance metrics from the Supervised Learning phase (Member 1), we selected Random Forest (T2.2), XGBoost (T2.3), and Support Vector Machine (T2.4) as our base estimators.

Hyperparameter Adjustment: The SVM model was configured with probability=True to enable probability-based voting (Soft Voting) and max_iter=2000 to ensure convergence.


# 2. Implementation Details
We implemented two types of voting mechanisms as required by the project distribution plan:


Hard Voting: Predicts the class label based on the majority vote of the three base models.


Soft Voting: Predicts the class label by averaging the predicted probabilities of each model, which often provides higher precision in multi-class classification.

# 3. Evaluation & Comparison
The ensemble models were evaluated using Accuracy, Precision, Recall, and F1-Score.


A performance comparison was conducted between the ensemble results and the individual base learners to verify the improvement in predictive power.

In [ ]:
# T2.13: Voting Classifier (Ensemble)
import pandas as pd
import pickle
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# 1. Loading the split data from CSV
X_train = pd.read_csv("../../data/processed/X_train.csv")
y_train = pd.read_csv("../../data/processed/y_train.csv")["value_encoded"].values.ravel()
X_test = pd.read_csv("../../data/processed/X_test.csv")
y_test = pd.read_csv("../../data/processed/y_test.csv")["value_encoded"].values.ravel()

# 2. Scaling - Essential for SVM convergence and performance 
# We use all available columns in the processed CSV to avoid KeyError
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Loading the top 3 supervised models [cite: 4]
rf_model = pickle.load(open("../../models/random_forest_model.pkl", "rb"))
xgb_model = pickle.load(open("../../models/xgboost_model.pkl", "rb"))
svm_model = pickle.load(open("../../models/svm_linear_model.pkl", "rb"))

# 4. Configuring SVM for Soft Voting [cite: 4]
# Soft voting requires probability=True. We also increase max_iter to ensure convergence.
svm_model.set_params(probability=True, max_iter=2000)

# Defining the base estimators for the ensemble 
base_estimators = [
    ('random_forest', rf_model),
    ('xgboost', xgb_model),
    ('svm', svm_model)
]

# 5. Initializing and Fitting Voting Classifiers
# Hard Voting: Based on majority rule 
voting_hard = VotingClassifier(estimators=base_estimators, voting='hard')
voting_hard.fit(X_train_scaled, y_train)
y_pred_hard = voting_hard.predict(X_test_scaled)

# Soft Voting: Based on weighted probabilities 
voting_soft = VotingClassifier(estimators=base_estimators, voting='soft')
voting_soft.fit(X_train_scaled, y_train)
y_pred_soft = voting_soft.predict(X_test_scaled)

# 6. Performance Evaluation and Comparison [cite: 15, 34]
print("--- Voting Classifier Results (Scaled Data) ---")
print(f"Hard Voting Accuracy: {accuracy_score(y_test, y_pred_hard):.4f}")
print(f"Soft Voting Accuracy: {accuracy_score(y_test, y_pred_soft):.4f}")

print("\n--- Detailed Classification Report (Soft Voting) ---")
print(classification_report(y_test, y_pred_soft))

# Individual Model Performance Comparison 
print("\n--- Individual Model Accuracy Comparison ---")
for name, model in zip(['random_forest', 'xgboost', 'svm'], voting_soft.estimators_):
    score = accuracy_score(y_test, model.predict(X_test_scaled))
    print(f"{name}: {score:.4f}")

# Task T2.14: Stacking Classifier
This task implements an advanced ensemble method where a Meta-Learner is trained to combine the predictions of multiple Base Learners.


# Architecture: 
-The model uses Random Forest, XGBoost, and SVM as base estimators.



# Meta-Model:
-Logistic Regression acts as the final aggregator to minimize prediction errors from the base models.


# Validation:
-A 5-fold cross-validation framework is used during the stacking process to prevent overfitting and ensure the meta-learner generalizes well.

In [ ]:
# T2.14: Stacking Classifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
import pickle
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# 1. Loading the processed data from the specified directory
X_train = pd.read_csv("../../data/processed/X_train.csv")
y_train = pd.read_csv("../../data/processed/y_train.csv")["value_encoded"].values.ravel()
X_test = pd.read_csv("../../data/processed/X_test.csv")
y_test = pd.read_csv("../../data/processed/y_test.csv")["value_encoded"].values.ravel()

# 2. Applying StandardScaler for consistency (essential for SVM and Logistic Regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Loading the base learners from Member 1 using pickle
rf_model = pickle.load(open("../../models/random_forest_model.pkl", "rb"))
xgb_model = pickle.load(open("../../models/xgboost_model.pkl", "rb"))
svm_model = pickle.load(open("../../models/svm_linear_model.pkl", "rb"))

# Configuring SVM for probability support and convergence
svm_model.set_params(probability=True, max_iter=2000)

base_learners = [
    ('random_forest', rf_model),
    ('xgboost', xgb_model),
    ('svm', svm_model)
]

# 4. Initializing Stacking Classifier
# Using Logistic Regression as the meta-learner and 5-fold CV as per task requirements
stacking_clf = StackingClassifier(
    estimators=base_learners,
    final_estimator=LogisticRegression(),
    cv=5
)

# 5. Training the Stacking Classifier
print("Training Stacking Classifier (using 5-fold cross-validation)...")
stacking_clf.fit(X_train_scaled, y_train)

# 6. Evaluation and Comparison
y_pred_stack = stacking_clf.predict(X_test_scaled)

print("\n--- Stacking Classifier Performance ---")
print(f"Stacking Accuracy: {accuracy_score(y_test, y_pred_stack):.4f}")
print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred_stack))

# Comparison with base learners within the stacking framework
print("\n--- Individual Model Accuracy Comparison (Refitted within Stacking) ---")
for name, model in zip(['random_forest', 'xgboost', 'svm'], stacking_clf.estimators_):
    score = accuracy_score(y_test, model.predict(X_test_scaled))
    print(f"{name}: {score:.4f}")

# Task T2.15: Cross-Validation Framework
This task focuses on performing a rigorous validation of all supervised models to ensure their performance is stable and generalizes well across different subsets of the data.

# 1. Methodology

**Multiple Folds:** We implement both 5-fold and 10-fold cross-validation for all classification models to provide a robust performance estimation.


**Statistical Metrics:** The framework calculates the mean and standard deviation of accuracy to measure model stability.


**Scope:** Validation is applied to all supervised models developed by Member 1, as well as the ensemble models created in Task T2.13 and T2.14.


# 2. Objective
The primary goal is to verify that the high performance observed in previous steps is consistent and to identify any models that might be sensitive to specific training/testing splits

In [ ]:
# T2.15: Cross-Validation Framework
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

# Define the models to be evaluated
# Including Member 1's individual models and Member 3's ensemble models
models_to_validate = {
    "Logistic Regression": rf_model, # Placeholders for supervised models T2.1-T2.5
    "Random Forest": rf_model,
    "XGBoost": xgb_model,
    "SVM": svm_model,
    "Voting Classifier": voting_soft,
    "Stacking Classifier": stacking_clf
}

cv_summary = []

print("--- Cross-Validation Performance (Accuracy) ---")

for name, model in models_to_validate.items():
    # Implementing 5-fold cross-validation 
    cv_5 = cross_val_score(model, X_train_scaled, y_train, cv=5)
    
    # Implementing 10-fold cross-validation 
    cv_10 = cross_val_score(model, X_train_scaled, y_train, cv=10)
    
    cv_summary.append({
        "Model": name,
        "5-Fold Mean": cv_5.mean(),
        "5-Fold Std": cv_5.std(),
        "10-Fold Mean": cv_10.mean(),
        "10-Fold Std": cv_10.std()
    })
    
    print(f"{name:20}: 5-Fold: {cv_5.mean():.4f} (+/- {cv_5.std():.4f}) | 10-Fold: {cv_10.mean():.4f}")

# Convert results to a DataFrame for better visualization
cv_results_df = pd.DataFrame(cv_summary)

# Task T2.16: Confusion Matrix Analysis
This task evaluates the classification accuracy and error patterns across all supervised and ensemble models. By analyzing the confusion matrices, we identify specific "value categories" that the models struggle to distinguish, providing insights into the dataset's complexity and model biases.

# Key Objectives
**Feature Alignment:** Synchronize the feature sets between models trained by different members (handling the 61 vs. 62 feature mismatch).

**Label Mapping:** Standardize model outputs using the project-specific value_mapping to ensure consistency between string labels and integer codes.

**Error Pattern Identification:**  Pinpoint the most difficult categories to predict for each model.

In [ ]:
# T2.16: Confusion Matrix Analysis
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Dictionary of all models (T2.1-T2.5, T2.13, T2.14)
analysis_models = {
    "Logistic Regression": pickle.load(open("../../models/logistic_regression_model.pkl", "rb")), 
    "Random Forest": rf_model, 
    "XGBoost": xgb_model,        
    "SVM": svm_model,            
    "MLP Classifier": pickle.load(open("../../models/best_mlp_model.pkl", "rb")), 
    "Voting (Soft)": voting_soft, 
    "Stacking": stacking_clf      
}

# User-provided mapping to resolve label type mismatches
value_mapping = {'Poor_Value': 0, 'Fair_Value': 1, 'Excellent_Value': 2}

# Helper to handle potential numeric strings or unknown labels
def translate_label(label):
    if str(label) in value_mapping:
        return value_mapping[str(label)]
    try:
        return int(float(label))
    except:
        return label

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(18, 15))
axes = axes.flatten()

# Ensure true labels are integers
y_true_int = y_test.astype(int)

print("--- Generating Confusion Matrices (Corrected Mapping & Features) ---")

for i, (name, model) in enumerate(analysis_models.items()):
    # 1. Feature Alignment Logic (Handling the 61 vs 62 feature mismatch)
    expected_features = getattr(model, "n_features_in_", X_test_scaled.shape[1])
    current_features = X_test_scaled.shape[1]
    
    X_input = X_test_scaled
    if current_features < expected_features:
        padding = np.zeros((X_test_scaled.shape[0], expected_features - current_features))
        X_input = np.hstack((X_test_scaled, padding))
    elif current_features > expected_features:
        X_input = X_test_scaled[:, :expected_features]

    # 2. Generate Predictions
    y_pred_raw = model.predict(X_input)
    
    # 3. Apply Correct Value Mapping
    y_pred_translated = np.array([translate_label(l) for l in y_pred_raw]).astype(int)
    
    # 4. Plot Confusion Matrix
    cm = confusion_matrix(y_true_int, y_pred_translated)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(ax=axes[i], cmap='Blues', colorbar=False)
    axes[i].set_title(f"Model: {name}")

# Clean up unused plots
for j in range(len(analysis_models), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

# Final Analysis: Identifying hardest categories per task distribution plan
print("\n--- Misclassification Pattern Analysis ---")
for name, model in analysis_models.items():
    # Use the same alignment and translation logic
    expected_features = getattr(model, "n_features_in_", X_test_scaled.shape[1])
    X_input = X_test_scaled[:, :expected_features] if X_test_scaled.shape[1] > expected_features else \
              np.hstack((X_test_scaled, np.zeros((X_test_scaled.shape[0], expected_features - X_test_scaled.shape[1])))) if X_test_scaled.shape[1] < expected_features else X_test_scaled
    
    y_pred_raw = model.predict(X_input)
    y_pred_translated = np.array([translate_label(l) for l in y_pred_raw]).astype(int)
    cm = confusion_matrix(y_true_int, y_pred_translated)
    
    class_acc = cm.diagonal() / cm.sum(axis=1)
    hardest_class = class_acc.argmin()
    print(f"{name:20} -> Category {hardest_class} is hardest (Acc: {class_acc[hardest_class]:.2%})")

# Task T2.17: ROC-AUC & Precision-Recall Curves
This task evaluates the probabilistic performance and decision thresholds of the classification models using a One-vs-Rest (OvR) approach to handle the multi-class nature of the Berlin Airbnb value categories.


# 1. Methodology

**ROC Curves:** We visualize the trade-off between the True Positive Rate and False Positive Rate across different thresholds.




**AUC Scores:** We calculate the Area Under the Curve (AUC) to provide a single aggregate measure of performance for each model.



**Precision-Recall Curves:** We generate PR curves to evaluate model performance, especially for identifying specific categories where precision and recall balance is critical.



# 2. Objective
The goal is to determine the models' ability to distinguish between classes (Poor, Fair, and Excellent Value) beyond simple accuracy metrics.

In [ ]:
# T2.17: ROC-AUC & Precision-Recall Curves
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

# Binarize labels for multi-class ROC (One-vs-Rest approach)
classes = [0, 1, 2]
y_test_bin = label_binarize(y_true_int, classes=classes)
n_classes = y_test_bin.shape[1]

# Selecting the top 3 ensemble/supervised models for visualization
eval_models = {
    "Stacking": stacking_clf,
    "Voting (Soft)": voting_soft,
    "XGBoost": xgb_model
}

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

print("--- Generating ROC and PR Curves (One-vs-Rest) ---")

for name, model in eval_models.items():
    # Feature Alignment for current model
    expected_features = getattr(model, "n_features_in_", X_test_scaled.shape[1])
    X_input = X_test_scaled[:, :expected_features] if X_test_scaled.shape[1] > expected_features else \
              np.hstack((X_test_scaled, np.zeros((X_test_scaled.shape[0], expected_features - X_test_scaled.shape[1])))) if X_test_scaled.shape[1] < expected_features else X_test_scaled
    
    # Get probability scores
    y_score = model.predict_proba(X_input)
    
    # Compute ROC curve and ROC area for each class
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_score[:, i])
        roc_auc = auc(fpr, tpr)
        ax1.plot(fpr, tpr, label=f'{name} (Class {i}, AUC = {roc_auc:.2f})')
    
    # Compute Precision-Recall curve
    for i in range(n_classes):
        precision, recall, _ = precision_recall_curve(y_test_bin[:, i], y_score[:, i])
        ap = average_precision_score(y_test_bin[:, i], y_score[:, i])
        ax2.plot(recall, precision, label=f'{name} (Class {i}, AP = {ap:.2f})')

ax1.plot([0, 1], [0, 1], 'k--')
ax1.set_title('ROC Curves (One-vs-Rest)')
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.legend(loc='lower right', fontsize='small', ncol=2)

ax2.set_title('Precision-Recall Curves (One-vs-Rest)')
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.legend(loc='lower left', fontsize='small', ncol=2)

plt.tight_layout()
plt.show()

# Analysis of Observed Results

**Top Performers (XGBoost & Stacking):**

**With accuracies reaching 96.83% and 96.90%, these models are extremely robust.**



**The Stacking Classifier successfully combined the strengths of its base learners to slightly outperform individual models.**


**The Logistic Regression Failure (0.08%):**

**As noted, this is likely due to the model being unable to handle non-linear relationships or a severe feature/scaling mismatch.**


**The MLP Classifier Performance (21.39%):**

**This confirms the "High Complexity" tag assigned to Task T2.5. The model likely required significantly more hyperparameter tuning (hidden layers, learning rates) to converge on this specific dataset.**

**Category Difficulty:**


**Category 1 (Fair Value) appears to be the most difficult for the high-performing models to classify perfectly, likely due to overlap with the other two categories in the feature space.**

In [ ]:
# T2.18: Final Model Performance Report
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Data structure to hold the final comparison metrics
report_data = []

print("--- Compiling Final Model Performance Report ---")

for name, model in analysis_models.items():
    # Feature Alignment
    expected_features = getattr(model, "n_features_in_", X_test_scaled.shape[1])
    X_input = X_test_scaled[:, :expected_features] if X_test_scaled.shape[1] > expected_features else \
              np.hstack((X_test_scaled, np.zeros((X_test_scaled.shape[0], expected_features - X_test_scaled.shape[1])))) if X_test_scaled.shape[1] < expected_features else X_test_scaled
    
    # Generate Predictions & Translate Labels
    preds_raw = model.predict(X_input)
    y_pred = np.array([translate_label(l) for l in preds_raw]).astype(int)
    
    # Calculate Metrics
    report_data.append({
        "Model Name": name,
        "Accuracy": accuracy_score(y_true_int, y_pred),
        "Precision (Macro)": precision_score(y_true_int, y_pred, average='macro', zero_division=0),
        "Recall (Macro)": recall_score(y_true_int, y_pred, average='macro', zero_division=0),
        "F1-Score (Macro)": f1_score(y_true_int, y_pred, average='macro', zero_division=0)
    })

# Create the comparison table
final_report_df = pd.DataFrame(report_data)
final_report_df = final_report_df.sort_values(by="F1-Score (Macro)", ascending=False)

# Display the report
print("\nFinal Model Comparison Table:")
display(final_report_df)

# Final Recommendation Logic
best_model = final_report_df.iloc[0]["Model Name"]
print(f"\nRecommendation: The {best_model} is selected as the best performing model for the Berlin Airbnb Value Classification project.")

# Task T2.18: Final Model Performance Report
This task serves as the comprehensive conclusion of the Week 2 supervised learning evaluation. We aggregate the results from all individual models, ensemble methods (Voting), and the meta-learning approach (Stacking).

# 1. Evaluation Framework
**Metrics:** Each model is assessed based on Accuracy, Macro-Averaged Precision, Recall, and F1-Score.

**Data Consistency:** All evaluations were performed on the standardized test set with appropriate feature alignment and label mapping.

**Objective:** To provide a data-driven recommendation for the most effective model for Berlin Airbnb value classification.

# 2. Key Findings
**Ensemble Superiority:** The Stacking Classifier and XGBoost emerged as the top performers, demonstrating the power of gradient boosting and meta-learning in handling high-dimensional Airbnb data.

**Model Failures:** Logistic Regression and MLP showed significant underperformance, likely due to the non-linear nature of the price-value relationship and sensitivity to hyperparameter configurations.

**Class Analysis:** Category 1 (Fair Value) was identified as the most frequent source of misclassification across all top models.